In [99]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA

In [100]:
# open final data
data = pd.read_csv('../../data/processed/final_data.csv')

# get the list of columns
columns = data.columns
positive_columns = [col for col in columns if '+' in col]
negative_columns = [col for col in columns if '-' in col]
original_columns = [col for col in columns if '+' not in col and '-' not in col]

In [101]:
# use only the training data
model_training_data = data[data['dataset'] == 'train']

# convert the datetime columns to datetime
model_training_data['datetime'] = pd.to_datetime(model_training_data['datetime'])

/var/folders/bn/prwtql5x5sg0mbkxc6508fqm0000gn/T/ipykernel_87480/545332924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_training_data['datetime'] = pd.to_datetime(model_training_data['datetime'])


In [102]:
test_period = 4
lag_period = 2
maximum_day = 20

# mask the data from day 3 to 15 of each month using 'datetime' column
negative_train_mask = model_training_data['datetime'].apply(lambda x: x.day > lag_period and x.day < maximum_day - test_period)
negative_test_mask = model_training_data['datetime'].apply(lambda x: x.day >= maximum_day - test_period)

# get the negative training data
negative_train_data = model_training_data[negative_train_mask][original_columns + negative_columns].copy()
negative_test_data = model_training_data[negative_test_mask][original_columns + negative_columns].copy()

# maske the data from day 5 to 17 of each month using 'datetime' column
positive_train_mask = model_training_data['datetime'].apply(lambda x: x.day > test_period and x.day < maximum_day - lag_period)
positive_test_mask = model_training_data['datetime'].apply(lambda x: x.day <= test_period)

# get the positive training data
positive_train_data = model_training_data[positive_train_mask][original_columns + positive_columns].copy()
positive_test_data = model_training_data[positive_test_mask][original_columns + positive_columns].copy()

# save the data into a dictionary
training_data = {
    'positive': {
        'X': positive_train_data,
        'y': positive_test_data
        },
    'negative': {
        'X': negative_train_data,
        'y': negative_test_data
        }
    }

In [103]:
training_data['negative']['X'].columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'dataset', 'count_original',
       'registered_original', 'casual_original', 'dayofyear', 'weekofyear',
       'dayofweek', 'windspeed, -1', 'atemp, -1', 'humidity, -1',
       'holiday, -1', 'workingday, -1', 'weather, -1', 'temp, -1',
       'casual, -1', 'registered, -1', 'count, -1', 'windspeed, -2',
       'atemp, -2', 'humidity, -2', 'holiday, -2', 'workingday, -2',
       'weather, -2', 'temp, -2', 'casual, -2', 'registered, -2', 'count, -2'],
      dtype='object')

In [104]:
# for each train or test assert there are no NaN values
assert not negative_train_data.isnull().values.any()
assert not negative_test_data.isnull().values.any()
assert not positive_train_data.isnull().values.any()
assert not positive_test_data.isnull().values.any()

In [105]:
def create_pipelines(target_columns, directions, rf=True, mlp=False, trees=100, max_iter=2000, hidden_layer_sizes=(100,)):
    # create a dictionary to store the pipelines
    pipelines = {}

    for target in target_columns:
        
        if 'original' in target:
            target_name = target[:-len('_original')]
        else:
            target_name = target

        for direction in directions:
                
            if rf:
                # create random forest pipeline for the target and direction
                globals()[f'{direction}_{target_name}_pipeline_rf'] = Pipeline([
                    ('scaler', StandardScaler()),
                    ('regressor', RandomForestRegressor(n_estimators=trees)),
                ])

                # save the pipeline
                pipelines[f'{direction}_{target_name}_pipeline_rf'] = globals()[f'{direction}_{target_name}_pipeline_rf']
            
            if mlp:
                # create MLP pipeline for the target and direction
                globals()[f'{direction}_{target_name}_pipeline_mlp'] = Pipeline([
                    ('scaler', StandardScaler()),
                    ('regressor', MLPRegressor(max_iter=max_iter, hidden_layer_sizes=hidden_layer_sizes, verbose=True)),
                ])

                # save the pipeline
                pipelines[f'{direction}_{target_name}_pipeline_mlp'] = globals()[f'{direction}_{target_name}_pipeline_mlp']

    return pipelines

In [106]:
def fit_pipelines(pipelines, train_data, target_columns, columns_not_to_use, directions, rf = True, mlp = False):

    print('\n\nFitting pipelines...')
    # create a dictionary to store the fitted pipelines
    fitted_pipelines = {}

    for target in target_columns:
        print(f'Fitting pipelines for {target}')
        
        if 'original' in target:
            target_name = target[:-len('_original')]
        else:
            target_name = target

        for direction in directions:

            df = train_data[direction]['X'].copy()
            target_data = df[target]
            drop_coumns = columns_not_to_use + target_columns
            df = df.drop(drop_coumns, axis=1)
            
            # get the pipelines
            if rf:
                pipeline_rf = pipelines[f'{direction}_{target_name}_pipeline_rf']
                
                pipeline_rf.fit(df, target_data)

                # save the fitted pipeline
                fitted_pipelines[f'{direction}_{target_name}_pipeline_rf'] = pipeline_rf

                """
                # create a dataframe to store the feature importances
                feature_importances = pd.DataFrame({
                    'feature': df.columns,
                    'importance': pipeline_rf.named_steps['regressor'].feature_importances_
                })

                # sort the features by importance
                feature_importances = feature_importances.sort_values('importance', ascending=False)

                # print the feature importances
                #print(f'{direction}_{target}_pipeline_rf')
                #print(feature_importances)
                """

            if mlp:
                pipeline_mlp = pipelines[f'{direction}_{target_name}_pipeline_mlp']
                
                # fit the pipeline
                pipeline_mlp.fit(df, target_data)

                # save the fitted pipeline
                fitted_pipelines[f'{direction}_{target_name}_pipeline_mlp'] = pipeline_mlp

    return fitted_pipelines

In [107]:
def store_prediction_with_lags(df, mask, direction, target, prediction):

    prediction_array = np.array(df[target])
    prediction_array[mask] = prediction

    #print('prediction array: ', prediction_array[:5], prediction_array[-5:], '\n')

    # store the prediction array in the dataframe
    df[target] = prediction_array

    # make the null values nan
    df[target] = df[target].apply(lambda x: x if x != 0 else np.nan)


    lags = [1, 2]
    sign = '-'

    if direction == 'positive':
        sign = '+'
        lags = [-1, -2]

    # get the 'datetime' column for the masked data
    datetime = df['datetime']
    #print('datetime: ', datetime[:5], datetime[-5:], '\n')
    datetime_masked = df[mask]['datetime']

    for lag in lags:
        #print('lag: ', lag)
        # get the lagged 'datetime' column
        lagged_datetime = datetime_masked + pd.DateOffset(days=lag)
        #print('lagged datetime: ', lagged_datetime[:5], lagged_datetime[-5:], '\n')
        #print('datetime masked: ', datetime_masked[:5], datetime_masked[-5:], '\n')

        # get the mask for lagged time
        lagged_mask = lagged_datetime.isin(datetime)
        #print('lagged mask: ', lagged_mask[:5], lagged_mask[-5:])
        #print('total lagged mask: ', lagged_mask.sum(), '\n')

        # get the mask for the lagged 'datetime' column
        datetime_mask =  datetime.isin(lagged_datetime)
        #print('datetime mask: ', datetime_mask[:5], datetime_mask[-5:])
        #print('total datetime mask: ', datetime_mask.sum(), '\n')

        #print('subset of lagged datetime: ', lagged_datetime[lagged_mask][:5], lagged_datetime[lagged_mask][-5:], '\n')
        #print('subset of datetime: ', datetime[datetime_mask][:5], datetime[datetime_mask][-24:18], '\n')

        # assert the number of elements in the lagged mask is equal to the number of elements in the datetime mask
        assert lagged_mask.sum() == datetime_mask.sum()

        if lagged_mask.sum() > 0:
            
            prediction_to_store = prediction[lagged_mask]
            #print('prediction to store: ', prediction_to_store[:5], prediction_to_store[-5:], '\n')
            # insert the prediction into the dataframe subsetted by the lagged mask
            column_to_insert = target + ', ' + sign + str(abs(lag))
            df.loc[datetime_mask, column_to_insert] = prediction_to_store

    return df

In [108]:
def predict_pipelines(fitted_pipelines, training_data, 
                      target_columns, columns_not_to_use, 
                      test_period, lag_period, maximum_day,
                      directions, rf=True, mlp=False):
    
    print('\nPredicting pipelines...')

    # create a dictionary to store the predictions
    predictions = {}
    dataframes = {}

    for target in target_columns:
        print(f'Predicting pipelines for {target}')

        if 'original' in target:
            target_name = target[:-len('_original')]
        else:
            target_name = target

        for direction in directions:

            # get the y data
            df = training_data[direction]['y'].copy()

            check_columns = ['casual', 'registered', 'count', 'casual_original', 'registered_original', 'count_original']
            additional_columns = [col for col in check_columns if col not in columns_not_to_use]
            drop_columns = columns_not_to_use + additional_columns

            # set the target column to NaN
            df[target] = np.nan

            if direction == 'negative':
                start_day = maximum_day - test_period
            elif direction == 'positive':
                start_day = test_period

            day = start_day
            days_predicted = 0

            while days_predicted < test_period:
                # mask the data to select all rows corresponding to a day of the month equal to start_day
                mask = df['datetime'].apply(lambda x: x.day == day)

                # get the data
                df_days = df[mask].copy()
                df_days = df_days.drop(drop_columns, axis=1)
                
                # get the pipeline
                if rf:
                    pipeline1 = fitted_pipelines[f'{direction}_{target_name}_pipeline_rf']
                if mlp:
                    pipeline2 = fitted_pipelines[f'{direction}_{target_name}_pipeline_mlp']

                # take the mean of the predictions if both pipelines are used
                if rf and mlp:
                    prediction1 = pipeline1.predict(df_days)
                    prediction2 = pipeline2.predict(df_days)
                    prediction = (prediction1 + prediction2) / 2
                elif rf:
                    print('drop columns: ', drop_columns)
                    print('columns: ', df_days.columns)
                    prediction = pipeline1.predict(df_days)
                elif mlp:
                    prediction = pipeline2.predict(df_days)

                # assert the lenght of the prediction is equal to the lenght of the mask
                assert len(prediction) == np.sum(mask)

                # store the prediction
                df = store_prediction_with_lags(df, mask, direction, target, prediction)
            
                if direction == 'negative':
                    day += 1

                elif direction == 'positive':
                    day -= 1

                days_predicted += 1

            prediction = df[target]
            
            if prediction.isna().sum() > 0:
                # print the dates with missing values
                print('Dates with missing values: ', df[df[target].isna()]['datetime'])

            # assert there are no missing values
            assert not prediction.isna().values.any()

            # store the predictions
            predictions[f'{direction}_{target_name}'] = prediction

            # store the dataframe
            dataframes[f'{direction}_{target_name}'] = df

    return predictions, dataframes

In [109]:
def evaluate_pipelines(predictions, training_data, target_columns, directions):
    # create a dictionary to store the evaluation metrics
    evaluation = pd.DataFrame()

    for target in target_columns:

        if 'original' in target:
            target_name = target[:-len('_original')]
        else:
            target_name = target

        for direction in directions:

            # get the y data
            y = training_data[direction]['y'][target]

            # get the predictions
            prediction = predictions[f'{direction}_{target_name}']

            # calculate the evaluation metrics
            mse = mean_squared_error(y, prediction)
            mae = mean_absolute_error(y, prediction)

            # save the evaluation metrics
            evaluation.loc[f'{direction}_{target_name}', 'mse'] = mse
            evaluation.loc[f'{direction}_{target_name}', 'mae'] = mae

    # print the evaluation metrics
    print(evaluation)

    return evaluation

In [110]:
columns_not_to_use = ['datetime', 'dataset', 'day', 'month', 'year']
smoothed_columns = ['casual', 'registered', 'count']
original_columns = ['casual_original', 'registered_original', 'count_original']

train_columns = smoothed_columns

if 'original' in train_columns[0]:
    additional_columns = str(col[:-len('_original')] for col in train_columns)
else:
    additional_columns = [str(col + '_original') for col in train_columns]
columns_not_to_use += additional_columns
print(columns_not_to_use)

test_columns = original_columns
directions = ['negative', 'positive']
rf = True
mlp = False
trees = 15
max_iter = 2000
hidden_layer_sizes = (100, 100)

# create the pipelines
pipelines = create_pipelines(train_columns, directions, 
                             rf=rf, mlp=mlp, trees=trees, 
                             max_iter = 2000, hidden_layer_sizes = (100, 100))

test_period = 4
lag_period = 2
maximum_day = 20

# fit the pipelines
fitted_pipelines = fit_pipelines(pipelines, training_data, 
                                 train_columns, columns_not_to_use,
                                 directions, rf=rf, mlp=mlp)

# predict the pipelines
predictions, dataframes = predict_pipelines(fitted_pipelines, training_data, 
                                test_columns, columns_not_to_use, 
                                test_period, lag_period, maximum_day,
                                directions, rf=rf, mlp=mlp)

# evaluate the pipelines
evaluation = evaluate_pipelines(predictions, training_data, test_columns, directions)

#TODO fix bugs with different train and test y

['datetime', 'dataset', 'day', 'month', 'year', 'casual_original', 'registered_original', 'count_original']


Fitting pipelines...
Fitting pipelines for casual
Fitting pipelines for registered
Fitting pipelines for count

Predicting pipelines...
Predicting pipelines for casual_original
drop columns:  ['datetime', 'dataset', 'day', 'month', 'year', 'casual_original', 'registered_original', 'count_original', 'casual', 'registered', 'count']
columns:  Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'hour', 'dayofyear', 'weekofyear', 'dayofweek',
       'windspeed, -1', 'atemp, -1', 'humidity, -1', 'holiday, -1',
       'workingday, -1', 'weather, -1', 'temp, -1', 'casual, -1',
       'registered, -1', 'count, -1', 'windspeed, -2', 'atemp, -2',
       'humidity, -2', 'holiday, -2', 'workingday, -2', 'weather, -2',
       'temp, -2', 'casual, -2', 'registered, -2', 'count, -2'],
      dtype='object')
drop columns:  ['datetime', 'dataset'

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- casual_original, -1
- casual_original, -2
